 Effects of Policy on the Housing Market ** 

In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [5]:
df = pd.read_csv("effects-of-policy-on-the-housing-market/train.csv")
df.head()


,Unnamed: 0.1,Unnamed: 0,id,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue
0,0,879,19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,...,4.80,4.82,4.90,4.87,4.69,4.81,f,3,0.77,2108
1,1,6416,1015650685503221866,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,...,NaN,NaN,NaN,NaN,NaN,NaN,f,1,NaN,2730
2,2,1820,35265562,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,...,4.90,4.78,4.97,4.94,4.90,4.75,f,1,3.22,2254
3,3,5346,911948980885194155,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,...,5.00,5.00,5.00,5.00,4.86,5.00,f,1,1.28,3187
4,4,2484,46069251,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,...,4.93,4.89,4.97,4.97,4.96,4.85,f,1,2.01,3479


In [6]:
import sys
print(sys.executable)

/Users/prajnanayak/WiDS Case Comp/myenv/bin/python


In [7]:

df.describe()

,Unnamed: 0.1,Unnamed: 0,id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,beds,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,monthly_revenue
count,5352.000000,5352.000000,5.352000e+03,5.352000e+03,5352.000000,5352.000000,5352.000000,5352.000000,5352.000000,5316.000000,...,4452.000000,4452.000000,4452.000000,4451.000000,4452.000000,4451.000000,4451.000000,5352.000000,4446.000000,5352.000000
mean,2675.500000,3354.300075,4.398265e+17,1.916746e+08,11.917601,18.213191,49.262454,-123.112037,3.519619,1.927389,...,4.778841,4.807729,4.760007,4.844812,4.867345,4.814383,4.662662,6.886584,1.825263,1837.484679
std,1545.133651,1926.579372,4.317929e+17,1.808806e+08,45.576743,61.783016,0.021200,0.037719,2.044174,1.180906,...,0.365107,0.352957,0.415887,0.314645,0.310635,0.295924,0.407893,19.657382,1.919462,1334.488315
min,0.000000,0.000000,1.318800e+04,4.662000e+03,1.000000,1.000000,49.202960,-123.217730,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.010000,0.000000
25%,1337.750000,1694.500000,3.365765e+07,2.637491e+07,1.000000,1.000000,49.249640,-123.130884,2.000000,1.000000,...,4.730000,4.780000,4.720000,4.810000,4.860000,4.760000,4.590000,1.000000,0.330000,798.750000
50%,2675.500000,3362.500000,5.987246e+17,1.318506e+08,2.000000,3.000000,49.268795,-123.115078,3.000000,2.000000,...,4.880000,4.900000,4.880000,4.940000,4.960000,4.900000,4.750000,1.000000,1.060000,1705.000000
75%,4013.250000,5016.250000,8.769719e+17,3.428838e+08,5.000000,8.000000,49.278994,-123.090373,4.000000,2.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.880000,3.000000,2.867500,2665.500000
max,5351.000000,6690.000000,1.044548e+18,5.505990e+08,466.000000,1037.000000,49.294360,-123.023730,16.000000,13.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,134.000000,15.820000,8584.000000


Preprocessiing Data

In [8]:
numerical_cols = ['host_listings_count', 'accommodates', 'beds', 'latitude', 'longitude', 
                  'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
                  'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
                  'review_scores_value', 'reviews_per_month']

categorical_cols = ['host_id', 'id']



In [9]:
preprocessor = make_column_transformer(
    (Pipeline([
        ('imputer', SimpleImputer(strategy='mean')), 
        ('scaler', StandardScaler())  
    ]), numerical_cols),
    
    (Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('onehot', OneHotEncoder(handle_unknown='ignore')) 
    ]), categorical_cols)
)

In [10]:
X = df.drop('monthly_revenue', axis=1)  
y = df['monthly_revenue']  

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('model', LinearRegression()) 
])

In [12]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['host_listings_count',
                                                   'accommodates', 'beds',
                                                   'latitude', 'longitude',
                                                   'review_scores_rating',
                                                   'review_scores_accuracy',
                                                   'review_scores_cleanliness',
                                                   'review_scores_checkin',
                                                   'review_scores_communication',
                                                   'review_scores_location',
                                                   'review_scores_value',
                                                   'reviews_per_month']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['host_id', 'id'])])),
                ('model', LinearRegression())])

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

test_df = pd.read_csv("effects-of-policy-on-the-housing-market/test.csv")

predictions = model_pipeline.predict(test_df)

# Create a new DataFrame with only 'id' and 'monthly_Revenue' prediction
result_df = pd.DataFrame({
    'id': test_df['id'],  # Assuming 'id' is a column in your test_df
    'monthly_Revenue': predictions
})

result_df.to_csv('monthly_revenue_predictions.csv', index=False)



# mse = mean_squared_error(y_val, y_pred)
# mae = mean_absolute_error(y_val, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error (MSE): {mse}')
# print(f'Mean Absolute Error (MAE): {mae}')
# print(f'R² Score: {r2}')

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('model', RandomForestRegressor())
])

# Define the hyperparameters to tune
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__bootstrap': [True, False]
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best model
print(f"Best hyperparameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(X_val)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END model__bootstrap=True, model__max_depth=None, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  17.5s
[CV] END model__bootstrap=True, model__max_depth=None, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  17.8s
[CV] END model__bootstrap=True, model__max_depth=None, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  18.0s
[CV] END model__bootstrap=True, model__max_depth=None, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  18.1s
[CV] END model__bootstrap=True, model__max_depth=None, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  18.3s
[CV] END model__bootstrap=True, model__max_depth=None, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=200; total time=  34.8s
[CV] END mod

KeyboardInterrupt: 